In [1]:
!pip install pandas as pd
!pip install numpy as np
!pip install re
!pip install json
!pip install os

ERROR: Could not find a version that satisfies the requirement as (from versions: none)
ERROR: No matching distribution found for as
You should consider upgrading via the 'C:\Users\prash\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


ERROR: Could not find a version that satisfies the requirement as (from versions: none)
ERROR: No matching distribution found for as
You should consider upgrading via the 'C:\Users\prash\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.
ERROR: Could not find a version that satisfies the requirement re (from versions: none)
ERROR: No matching distribution found for re
You should consider upgrading via the 'C:\Users\prash\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.
ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json
You should consider upgrading via the 'C:\Users\prash\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.
ERROR: Could not find a version that satisfies the requirement os (from versions: none)
ERROR: No matching distribution found for os
You should consider upgrading vi

In [2]:
!pip install chromadb
!pip install sentence-transformers
!pip install transformers torch

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Using cached httpx-0.28.1-py3-none-any.whl (73 kB)
  Using cached importlib_metadata-8.7.0-py3-none-any.whl (27 kB)
  Using cached tomli-2.2.1-py3-none-any.whl (14 kB)
  Using cached pyproject_hooks-1.2.0-py3-none-any.whl (10 kB)
  Using cached httpcore-1.0.9-py3-none-any.whl (78 kB)
  Using cached h11-0.16.0-py3-none-any.whl (37 kB)
  Using cached zipp-3.23.0-py3-none-any.whl (10 kB)
  Using cached shellingham-1.5.4-py2.py3-none-any.whl (9.8 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl (10 kB)
  Created wheel for pypika: filename=pypika-0.48.9-py2.py3-none-any.whl size=53916 sha256=3eb45ed61477069721e2d3e186ad3425363bfc3566e241d32894486c10091a9e
  Stored in directory: c:\

You should consider upgrading via the 'C:\Users\prash\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\prash\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\prash\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


##### 1️⃣ Install required packages (run once in your environment)
##### pip install pandas sentence-transformers transformers torch chromadb

In [3]:
import pandas as pd
import re
import json
import numpy as np
import chromadb
from chromadb.utils import embedding_functions
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import torch

c:\Users\prash\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# --------------------------
# 2️⃣ Load and clean CSV
# --------------------------
df = pd.read_csv("Constitution Of India.csv")
text_col = "Articles"

def clean_text(t):
    if pd.isna(t):
        return ""
    s = str(t)
    s = s.replace("\n", " ").replace("\t", " ")
    s = re.sub(r"\s+", " ", s)
    s = re.sub(r"http\S+|www\.\S+", " ", s)
    s = re.sub(r"\S+@\S+", " ", s)
    return s.strip()

df["clean_text"] = df[text_col].apply(clean_text)
df = df[df["clean_text"].str.len() > 30]

In [6]:
# --------------------------
# 3️⃣ Chunk the text
# --------------------------
def chunk_text(text, max_chars=1000, overlap=200):
    chunks = []
    start = 0
    while start < len(text):
        end = min(start + max_chars, len(text))
        chunks.append(text[start:end])
        if end == len(text):
            break
        start = end - overlap
    return chunks

all_chunks = []
for row in df["clean_text"]:
    all_chunks.extend(chunk_text(row))

print("Total chunks:", len(all_chunks))

# Save chunks locally (optional)
with open("chunks_data.json", "w", encoding="utf-8") as f:
    json.dump(all_chunks, f, indent=2, ensure_ascii=False)

Total chunks: 698


In [7]:
# --------------------------
# 4️⃣ Embed chunks
# --------------------------
embedder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
embeddings = embedder.encode(all_chunks, convert_to_numpy=True, show_progress_bar=True)
embeddings = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)  # normalize
np.save("embeddings.npy", embeddings)
print("✅ Chunks and embeddings saved!")

c:\Users\prash\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\prash\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not insta

✅ Chunks and embeddings saved!


In [8]:
# --------------------------
# 5️⃣ Upload to ChromaDB (free-tier safe)
# --------------------------
client = chromadb.CloudClient(
  api_key='ck-8ogrFSdCtYH9Bxh9mZ7A4dMsGbdQ2zp3VrLKNeHrfirE',
  tenant='f9ac33d4-f162-4b85-b38d-1c8b719366e6',
  database='VECDB'
)

collection_name = "constitution_articles"
if collection_name in [c.name for c in client.list_collections()]:
    collection = client.get_collection(name=collection_name)
else:
    collection = client.create_collection(name=collection_name)

MAX_RECORDS = 300  # free-tier limit
batch_chunks = all_chunks[:MAX_RECORDS]
batch_embeddings = embeddings[:MAX_RECORDS].tolist()
batch_ids = [str(i) for i in range(len(batch_chunks))]
batch_metadatas = [{"source": f"chunk_{i}"} for i in range(len(batch_chunks))]

collection.add(
    documents=batch_chunks,
    embeddings=batch_embeddings,
    ids=batch_ids,
    metadatas=batch_metadatas
)
print(f"✅ Added {len(batch_chunks)} chunks to ChromaDB!")

✅ Added 300 chunks to ChromaDB!


In [9]:
# --------------------------
# 6️⃣ Load FLAN-T5 for QA (PyTorch, no ONNX)
# --------------------------
model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
device = 0 if torch.cuda.is_available() else -1

qa_pipeline = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    device=device,
    framework="pt"  # force PyTorch
)

c:\Users\prash\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\prash\.cache\huggingface\hub\models--google--flan-t5-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back 

In [10]:
# --------------------------
# 7️⃣ Query functions
# --------------------------
def retrieve(query, top_k=5):
    results = collection.query(query_texts=[query], n_results=top_k)
    docs = results.get('documents', [[]])[0]  # safe access
    if not docs:
        return ["No relevant context found."]
    return docs

def answer_question(question, top_k=5):
    retrieved = retrieve(question, top_k)
    context = "\n\n".join(retrieved)
    prompt = f"Context: {context}\n\nQuestion: {question}\n\nAnswer:"
    result = qa_pipeline(prompt, max_length=256, do_sample=False)
    return result[0]["generated_text"], retrieved

In [12]:
# --------------------------
# 8️⃣ Interactive QA loop
# --------------------------
print("\n✅ Constitution QA system ready! Type your question (type 'exit' to quit).")
while True:
    q = input("\nQuestion: ")
    if q.lower() in ["exit", "quit"]:
        break
    ans, ctx = answer_question(q)
    print("\nAnswer:", ans)
    print("\n--- Retrieved Context ---")
    for c in ctx:
        print(c[:300], "...\n")


✅ Constitution QA system ready! Type your question (type 'exit' to quit).


C:\Users\prash\.cache\chroma\onnx_models\all-MiniLM-L6-v2\onnx.tar.gz: 100%|██████████| 79.3M/79.3M [01:21<00:00, 1.02MiB/s]
Token indices sequence length is longer than the specified maximum sequence length for this model (636 > 512). Running this sequence through the model will result in indexing errors



Answer: All laws in force in the territory of India immediately before the commencement of this Constitution and not previously repealed

--- Retrieved Context ---
147. Interpretation In this Chapter and in Chapter V of Part VI references to any substantial question of law as to the interpretation of this Constitution shall be construed as including references to any substantial question of law as to the interpretation of the Government of India Act, 1935 (inc ...

ory of India before the commencement of this Constitution and not previously repealed, notwithstanding that any such law or any part thereof may not be then in operation either at all or in particular areas (4) Nothing in this article shall apply to any amendment of this Constitution made under Arti ...

13. Laws inconsistent with or in derogation of the fundamental rights (1) All laws in force in the territory of India immediately before the commencement of this Constitution, in so far as they are inconsistent with the pro